In [23]:
import os
import shutil
import random
import itertools

# Changeable params

In [14]:
# Should be the directory that contains subdirectories of the aves dataset
# TODO: Change this as necessary
full_dir = r'D:\Resources\Inat_Partial\Aves_Small_SS2'
# Make sure to clean these two directories first
train_dir = r'D:\Resources\Inat_Partial\Aves_Small_SS2_Train'
val_dir = r'D:\Resources\Inat_Partial\Aves_Small_SS2_Validation'

random_seed = 5703
# default is 5-fold stratified validation
k = 5
# Whether to shuffle the file first before splitting them, 
# by default always use the last 10% of the images in each directory as validation
split_shuffle = False

In [3]:
if(os.path.isdir(train_dir) == False):
    os.mkdir(train_dir)
if(os.path.isdir(val_dir) == False):
    os.mkdir(val_dir)

In [19]:
def chunks(seq, num):
    """
    Divide a sequence into num of chunks
    https://stackoverflow.com/questions/2130016/splitting-a-list-into-n-parts-of-approximately-equal-length
    """
    avg = len(seq) / float(num)
    out = []
    last = 0.0

    while last < len(seq):
        out.append(seq[int(last):int(last + avg)])
        last += avg

    return out

In [39]:
subdirs = os.listdir(full_dir)
# make the subdir for cross validation first, since the loop order is different
for cur_k in range(k):
    cur_train_subdir = os.path.join(train_dir, r'CV_'+str(cur_k))
    cur_val_subdir = os.path.join(val_dir, r'CV_'+str(cur_k))
    os.mkdir(cur_train_subdir)
    os.mkdir(cur_val_subdir)
for subdir in subdirs:
    cur_parent_subdir = os.path.join(full_dir, subdir)
    parent_subdir_images = os.listdir(cur_parent_subdir)
    if split_shuffle == True:
        random.seed(random_seed)
        random.shuffle(parent_subdir_images)
    
    # Split the subdir images into even sized k chunks
    parent_subdir_images = list(chunks(parent_subdir_images, k))
    for cur_k in range(k):
        cur_train_subdir = os.path.join(train_dir, r'CV_'+str(cur_k), subdir)
        cur_val_subdir = os.path.join(val_dir, r'CV_'+str(cur_k), subdir)
        os.mkdir(cur_train_subdir)
        os.mkdir(cur_val_subdir)
        # Joining list of lists
        train_images = (list(itertools.chain(*parent_subdir_images[0:cur_k])) + 
                        list(itertools.chain(*parent_subdir_images[cur_k+1:k])))
        val_images = parent_subdir_images[cur_k]
        for img_to_train in train_images:
            shutil.copy(os.path.join(cur_parent_subdir,img_to_train), os.path.join(cur_train_subdir,img_to_train))
        for img_to_val in val_images:
            shutil.copy(os.path.join(cur_parent_subdir,img_to_val), os.path.join(cur_val_subdir,img_to_val))
    